In [1]:
import numpy as np
import pandas as pd
import sys
import tensorflow as tf
import tensorflow.contrib.layers as layers
import keras
from tqdm import tqdm, tnrange, tqdm_notebook
import math

Using TensorFlow backend.


In [2]:
tf.__version__

'1.0.1'

In [3]:
#Read data(CIFAR10)
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [4]:
#Setting hyperparameters
img_row = 32
img_col = 32
img_ch = 3
num_class = 10

batch_size = 128
baseLR = 0.0001
maxLR = 0.0006
num_epoch = 20

In [5]:
#label data --> one-hot encoding
y_train = keras.utils.np_utils.to_categorical(y_train,num_class)
y_test = keras.utils.np_utils.to_categorical(y_test,num_class)

In [6]:
X = tf.placeholder('float32',shape=[None,img_row,img_col,img_ch],name="input")
Y = tf.placeholder('float32',shape=[None,num_class])
lr = tf.placeholder('float32',shape=[])

In [7]:
def conv_net(_input):
    
    #conv layer 1
    with tf.variable_scope('conv1') as scope:
        W = tf.get_variable('weight',[3,3,3,32],initializer=layers.xavier_initializer())
        b = tf.get_variable('bias',[32],initializer=layers.xavier_initializer())
        conv1 = tf.nn.conv2d(_input,W,strides=[1,1,1,1],padding='SAME')
        conv1 = tf.nn.relu(conv1+b,name=scope.name)
        print( conv1.shape)
        
    #conv layer2
    with tf.variable_scope('conv2') as scope:
        W = tf.get_variable('weight',[3,3,32,32],initializer=layers.xavier_initializer())
        b = tf.get_variable('bias',[32],initializer=layers.xavier_initializer())
        conv2 = tf.nn.conv2d(conv1,W,strides=[1,1,1,1],padding='SAME')
        conv2 = tf.nn.relu(conv2+b)
        conv2 = tf.nn.max_pool(conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
        dr1 = tf.nn.dropout(conv2,keep_prob=0.25,name=scope.name)
        print(dr1.shape)
        
    #conv layer3
    with tf.variable_scope('conv3') as scope:
        W = tf.get_variable('weight',[3,3,32,64],initializer=layers.xavier_initializer())
        b = tf.get_variable('bias',[64],initializer=layers.xavier_initializer())
        conv3 = tf.nn.conv2d(dr1,W,strides=[1,1,1,1],padding='SAME')
        conv3 = tf.nn.relu(conv3+b)
        print(conv3.shape)
        
    #conv layer4
    with tf.variable_scope('conv4') as scope:
        W = tf.get_variable('weight',[3,3,64,64],initializer=layers.xavier_initializer())
        b = tf.get_variable('bias',[64],initializer=layers.xavier_initializer())
        conv4 = tf.nn.conv2d(conv3,W,strides=[1,1,1,1],padding='SAME')
        conv4 = tf.nn.relu(conv4+b)
        conv4 = tf.nn.max_pool(conv4,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
        dr2 = tf.nn.dropout(conv4,keep_prob=0.25,name=scope.name)
        print(dr2.shape)
        
    #FC1
    with tf.variable_scope('fc1') as scope:
        dense = tf.reshape(dr2,[-1,4096])
        print(dense.shape)
        W = tf.get_variable('weight',[4096,512],initializer=layers.xavier_initializer())
        b = tf.get_variable('bias',[512],initializer=layers.xavier_initializer())
        
        fc1 = tf.nn.relu(tf.matmul(dense,W)+b)
        dr3 = tf.nn.dropout(fc1,keep_prob=0.5)
        print(dr3.shape)
   
    #FC2
    with tf.variable_scope('fc2') as scope:
        W = tf.get_variable('weight',[512,10],initializer=layers.xavier_initializer())
        b = tf.get_variable('bias',[10],initializer=layers.xavier_initializer())
        
        fc2 = tf.nn.relu(tf.matmul(dr3,W)+b)
        print(fc2.shape)
        pred = fc2
        
    return pred


In [8]:
with tf.device('/gpu:0'):
    pred = conv_net(X)

(?, 32, 32, 32)
(?, 16, 16, 32)
(?, 16, 16, 64)
(?, 8, 8, 64)
(?, 4096)
(?, 512)
(?, 10)


In [9]:
with tf.device('/cpu:1'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=pred))
    

In [10]:
with tf.device('/cpu:2'):
    optimizer = tf.train.RMSPropOptimizer(learning_rate=lr).minimize(loss)
    correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred,'float32'))

In [11]:
def CLA_triangular(baseLR,iteration,stepsize):
    cycle = math.floor(1+iteration/(2*stepsize))
    x = math.fabs(iteration/stepsize - 2*cycle + 1)
    baseLR = baseLR +(maxLR - baseLR)*max(0,(1-x))
    return baseLR

In [12]:
#setting tensorflow config
config = tf.ConfigProto(allow_soft_placement=True,log_device_placement=True,device_count = {'GPU': 1})
config.gpu_options.per_process_gpu_memory_fraction = 0.6

In [13]:
saver = tf.train.Saver()

In [14]:
def optimization(X_train,Y_train,X_valid,Y_valid,model_dir,patience=-1):
    
    check_point = 0
    best_validation_accuracy = 0.0
    
    for epoch in tqdm_notebook(range(num_epoch),desc="epoch"):
        total_batch = int(np.shape(X_train)[0]/batch_size)
        
        #data shuffling
        X_train_copy = np.copy(X_train)
        Y_train_copy = np.copy(Y_train)
        
        train_index = np.arange(X_train.shape[0])
        np.random.shuffle(train_index)
        
        X_train_copy = X_train_copy[train_index]
        Y_train_copy = Y_train_copy[train_index]
        
        for i in range(total_batch):
            
            batch_x = X_train_copy[i*batch_size:(i+1)*batch_size]
            batch_y = Y_train_copy[i*batch_size:(i+1)*batch_size]
            
            iterator_size = (epoch)*X_train.shape[0]+(i+1)
            batch_lr = CLA_triangular(baseLR,iterator_size,total_batch/2)
            
            feed_dict_train = {X:batch_x, Y:batch_y, lr:batch_lr}
            
            sess.run(optimizer,feed_dict=feed_dict_train)
            
            acc_train = sess.run(accuracy,feed_dict=feed_dict_train)
            
            sys.stdout.write("\r%0.3f"%acc_train)
            
            if((i+1) % total_batch ==0):
                acc_train = sess.run(accuracy,feed_dict=feed_dict_train)
                acc_valid = sess.run(accuracy,feed_dict={X:X_valid,Y:Y_valid,lr:batch_lr})
                
                if(best_validation_accuracy < acc_valid):
                    best_validation_accuracy = acc_valid
                    
                    improved_str = '*'
                    checkpoint = 0
                    saver.save(sess,model_dir)
                
                else:
                    improved_str = ''
                    checkpoint +=1
                
                msg = "epoch: {0:>6}, Train-Batch Accuracy: {1:6.1%}, Validation Accuracy: {2:6.1%} {3} "
                print(msg.format(epoch+1,acc_train,acc_valid,improved_str))
        if(checkpoint ==patience):
            print("No improvement found. Early stopping")
            break

In [15]:
saver = tf.train.Saver()

In [16]:
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    optimization(x_train,y_train,x_test,y_test,"model/2.CNN_tutorial_bst_model")

0.125


KeyboardInterrupt: 